In [1]:
import numpy as np
import pandas as pd
import csv
import datetime as dt
from sqlalchemy import create_engine

# Extract

### Extract CSVs into DataFrames

In [2]:
#IMDB database
imdb_path = "resources/imdb_top_1000.csv"
#Acadeny Awards database
academy_award_path = "resources/academy_awards.csv"

imdb_df = pd.read_csv(imdb_path)
academy_award_df = pd.read_csv(academy_award_path)

# Transform

### Transforming IMDB dataframe

### Transform Academy Award dataframe

In [3]:
print(f"Number of rows in Academy Award table: {len(academy_award_df.index)}")
academy_award_df.head()

Number of rows in Academy Award table: 9964


,Year,Ceremony,Award,Winner,Name,Film
0,1927/1928,1,Actor,NaN,Richard Barthelmess,The Noose
1,1927/1928,1,Actor,1.0,Emil Jannings,The Last Command
2,1927/1928,1,Actress,NaN,Louise Dresser,A Ship Comes In
3,1927/1928,1,Actress,1.0,Janet Gaynor,7th Heaven
4,1927/1928,1,Actress,NaN,Gloria Swanson,Sadie Thompson


In [4]:
# Selecting columns in academy award dataframe
academy_award_df2 = academy_award_df[['Award','Winner','Name','Film']]

# drop empty film & Name cells
academy_award_df2 = academy_award_df2.dropna(subset=['Film'])
academy_award_df2 = academy_award_df2.dropna(subset=['Name'])

# drop empty winner cells as they are only nominees 
academy_award_df2 = academy_award_df2.dropna(subset=['Winner'])

# # dropping times where a movie is credited with an award twice (for example, a movie wins best cinematography, and there's 2 cinematographers)
academy_award_df2 = academy_award_df2.drop_duplicates(subset=['Award','Film'])
academy_award_df2 = academy_award_df2.drop_duplicates(subset=['Award','Name'])


print(f"Number of rows in Clean Academy Award table: {len(academy_award_df2.index)}")
academy_award_df2.head()


Number of rows in Clean Academy Award table: 1775


,Award,Winner,Name,Film
1,Actor,1.0,Emil Jannings,The Last Command
3,Actress,1.0,Janet Gaynor,7th Heaven
6,Art Direction,1.0,William Cameron Menzies,The Dove; Tempest
9,Cinematography,1.0,Charles Rosher,Sunrise
11,Directing (Comedy Picture),1.0,Lewis Milestone,Two Arabian Knights


### Creating ERD Tables
Academy award table

In [5]:
#movies table
movies_table=imdb_df[["Series_Title","Released_Year","Certificate","Runtime","Genre","IMDB_Rating","Meta_score","Director","Gross"]]
print(f"Number of rows in movies table: {len(movies_table.index)}")
movies_table.head()

Number of rows in movies table: 1000


,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Meta_score,Director,Gross
0,The Shawshank Redemption,1994,A,142 min,Drama,9.3,80.0,Frank Darabont,"28,341,469"
1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,100.0,Francis Ford Coppola,"134,966,411"
2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,84.0,Christopher Nolan,"534,858,444"
3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,90.0,Francis Ford Coppola,"57,300,000"
4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,96.0,Sidney Lumet,"4,360,000"


In [6]:
#directors table
#all directors who have directed movies in the IMDB top 1000
directors=pd.DataFrame(movies_table["Director"].unique())
directors.rename(columns={0:"Director_Name"})
print(f"Number of rows in Directors table: {len(directors.index)}")
directors.head()

Number of rows in Directors table: 548


,0
0,Frank Darabont
1,Francis Ford Coppola
2,Christopher Nolan
3,Sidney Lumet
4,Peter Jackson


In [7]:
#directed_by table
#all movies 
directored_by=movies_table[["Series_Title","Director"]]
directored_by=directored_by.rename(columns={"Director":"Director_Name"})
print(f"Number of rows in Directored by table: {len(directored_by.index)}")
directored_by.head()

Number of rows in Directored by table: 1000


,Series_Title,Director_Name
0,The Shawshank Redemption,Frank Darabont
1,The Godfather,Francis Ford Coppola
2,The Dark Knight,Christopher Nolan
3,The Godfather: Part II,Francis Ford Coppola
4,12 Angry Men,Sidney Lumet


### More Transformning: Only selecting movies from the top 1000 list that have won an adademy award

In the academy awards table, some movies appear in the "Film" column whilst other movies appear in the "Name" column.

In [8]:
#Merging IMDB table with Academy Award table so only academy awards to top 1000 movies remain
#Some film titles are in the Film column of Academy Award table, these are merged here

oscar_on_film=pd.merge(academy_award_df2,movies_table,how='left',left_on="Film",right_on="Series_Title")
oscar_on_film=oscar_on_film.dropna(subset=["IMDB_Rating"])
oscar_on_film=oscar_on_film.drop(columns=["Film","Name"])
print(f"Number of rows in oscar_on_film table: {len(oscar_on_film.index)}")
oscar_on_film.head()

Number of rows in oscar_on_film table: 123


,Award,Winner,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Meta_score,Director,Gross
12,Special Award,1.0,The Circus,1928,Passed,72 min,"Comedy, Romance",8.1,90.0,Charles Chaplin,NaN
24,Directing,1.0,All Quiet on the Western Front,1930,U,152 min,"Drama, War",8.0,91.0,Lewis Milestone,"3,270,000"
60,Actor,1.0,It Happened One Night,1934,Approved,105 min,"Comedy, Romance",8.1,87.0,Frank Capra,"4,360,000"
61,Actress,1.0,It Happened One Night,1934,Approved,105 min,"Comedy, Romance",8.1,87.0,Frank Capra,"4,360,000"
122,Special Award,1.0,A Star Is Born,2018,UA,136 min,"Drama, Music, Romance",7.6,88.0,Bradley Cooper,"215,288,866"


In [9]:
#some film titles are in the Name column of Academy Award table, these are merged here

oscar_on_name=pd.merge(academy_award_df2,movies_table,how="left",left_on="Name",right_on="Series_Title")
oscar_on_name=oscar_on_name.dropna(subset=["IMDB_Rating"])
oscar_on_name=oscar_on_name.drop(columns=["Film","Name"])
print(f"Number of rows in oscar_on_name table: {len(oscar_on_name.index)}")
oscar_on_name.head()

Number of rows in oscar_on_name table: 12


,Award,Winner,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Meta_score,Director,Gross
1758,Animated Feature Film,1.0,Inside Out,2015,U,95 min,"Animation, Adventure, Comedy",8.1,94.0,Pete Docter,"356,461,711"
1759,Costume Design,1.0,Mad Max: Fury Road,2015,UA,120 min,"Action, Adventure, Sci-Fi",8.1,90.0,George Miller,"154,058,340"
1762,Film Editing,1.0,Mad Max: Fury Road,2015,UA,120 min,"Action, Adventure, Sci-Fi",8.1,90.0,George Miller,"154,058,340"
1763,Makeup and Hairstyling,1.0,Mad Max: Fury Road,2015,UA,120 min,"Action, Adventure, Sci-Fi",8.1,90.0,George Miller,"154,058,340"
1764,Music (Original Score),1.0,The Hateful Eight,2015,A,168 min,"Crime, Drama, Mystery",7.8,68.0,Quentin Tarantino,"54,117,416"


### More ERD tables

In [10]:
#oscar_winners table
#both dataframes above are merged for a complete list of academy awards given to films in iMBD top 1000

oscar_winners=pd.merge(oscar_on_film,oscar_on_name,how='outer')
oscar_winners=oscar_winners.drop(columns=["Certificate","Winner","Released_Year","Runtime","Genre","IMDB_Rating","Meta_score","Director","Gross"])
print(f"Number of rows in oscar_on_name  table: {len(oscar_winners.index)}")
oscar_winners.head()

Number of rows in oscar_on_name  table: 135


,Award,Series_Title
0,Special Award,The Circus
1,Directing,All Quiet on the Western Front
2,Actor,It Happened One Night
3,Actress,It Happened One Night
4,Special Award,A Star Is Born


In [11]:
# oscar_categories table
# all unique award categories from the academy award dataframe

oscar_categories_df = pd.DataFrame(academy_award_df2['Award'].unique())
oscar_categories_df=oscar_categories_df.rename(columns={0:"Category Name"})
print(f"Number of rows in oscar_categories_df table: {len(oscar_categories_df.index)}")
oscar_categories_df.head()

Number of rows in oscar_categories_df table: 105


,Category Name
0,Actor
1,Actress
2,Art Direction
3,Cinematography
4,Directing (Comedy Picture)


### Creating ERD tables 

IMDB tables

In [12]:
imdb_df_copy = imdb_df.copy()

In [13]:
## create movie and genre table

genre_list = imdb_df_copy['Genre'].str.split(',')
movie_genres = pd.DataFrame({'Series_Title': imdb_df_copy['Series_Title'], 'Genre': genre_list})
len_list = list(map(len, movie_genres['Genre']))
movie_genres = pd.DataFrame({'Series_Title': np.repeat(movie_genres['Series_Title'], len_list), 'Genre': np.concatenate(movie_genres['Genre'].values)})
#striping leading and ending spaces in genre
movie_genres["Genre"] = movie_genres["Genre"].str.strip()
#### movie and genres table
print(f"Number of rows in movie_genres table: {len(movie_genres.index)}")
movie_genres.head()


Number of rows in movie_genres table: 2541


,Series_Title,Genre
0,The Shawshank Redemption,Drama
1,The Godfather,Crime
1,The Godfather,Drama
2,The Dark Knight,Action
2,The Dark Knight,Crime


In [14]:
# create genres table
genre = pd.DataFrame({'Genre': movie_genres['Genre'].unique()})
#### genre table
print(f"Number of rows in genre  table: {len(genre.index)}")
genre.head()

Number of rows in genre  table: 21


,Genre
0,Drama
1,Crime
2,Action
3,Adventure
4,Biography


In [17]:
# create movies table
movies = imdb_df_copy[['Series_Title', 'Runtime', 'Released_Year', 'IMDB_Rating', 'Meta_score', 'Gross']].copy()
#transform Runtime column as int
movies['Runtime'] = movies['Runtime'].str.replace(' min', '').astype(int)
# transform gross column: remove commas, fillna 
movies['Gross'] = movies['Gross'].str.replace(',', '').fillna(0)
# transform gross column: convert to int
movies['Gross'] = movies['Gross'].astype(int)
# handle errors in released year colunn, transform to datetime
movies['Released_Year'] = pd.to_numeric(movies['Released_Year'], errors='coerce', downcast='integer').fillna(0)
#### movies table
print(f"Number of rows in movies  table: {len(movies.index)}")
movies.head()


Number of rows in movies  table: 1000


,Series_Title,Runtime,Released_Year,IMDB_Rating,Meta_score,Gross
0,The Shawshank Redemption,142,1994.0,9.3,80.0,28341469
1,The Godfather,175,1972.0,9.2,100.0,134966411
2,The Dark Knight,152,2008.0,9.0,84.0,534858444
3,The Godfather: Part II,202,1974.0,9.0,90.0,57300000
4,12 Angry Men,96,1957.0,9.0,96.0,4360000


# LOAD